In [1]:
#import packages
import torch
import torch.nn as nn
import torch.nn.functional as f
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

In [2]:
#utils
import numpy as np
import time
import copy

In [4]:
#Model Implement
class basicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
    
        #BatchNorm inclues Bias so set bias = false on conv2d
        self.residual_func = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride = stride, padding =1 , bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels* basicBlock.expansion, kernel_size=3, stride=1, padding=1, bias= False),
            nn.BatchNorm2d(out_channels*basicBlock.expansion)
        )
    
        #identity mapping: using when input's and output's feature map size and numbers of filters are same
        self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()
    
        #Projection mapping using 1x1 Conv
        if stride != 1 or in_channels != basicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * basicBlock.expansion, kernel_size=1, stride=stride, bias =False),
                nn.BatchNorm2d(out_channels * basicBlock.expansion)
            )
    
    def forward(self, x):
        x = self.residual_func(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [27]:
class bottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.residual_func = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias =False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * bottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * bottleNeck.expansion)
        )
        
        self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()
        
        if stride != 1 or in_channels != out_channels * bottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * bottleNeck.expansion, kernel_size=1, stride=stride, bias = False),
                nn.BatchNorm2d(out_channels * bottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_func(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [28]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes = 10, init_weights = True):
        super().__init__()
        
        self.in_channels = 64
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.conv2_x = self._make_layer(block, 64, num_blocks[0],1)
        self.conv3_x = self._make_layer(block, 128, num_blocks[1],2)
        self.conv4_x = self._make_layer(block, 256, num_blocks[2],2)
        self.conv5_x = self._make_layer(block, 512, num_blocks[3],2)
        
        self.avg_pool= nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        #weight initialization
        if init_weights:
            self._initialize_weights()
            
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride]+[1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    #define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias,0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.constant_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)
                    
    def resnet18():
        return ResNet(basicBlock, [2,2,2,2])
    
    def resnet34():
        return ResNet(basicBlock, [3,4,6,3])
    
    def resnet50():
        return ResNet(bottleNeck, [3,4,6,3])
    
    def resnet101():
        return ResNet(bottleNeck, [3,4,23,3])
    
    def resnet152():
        return ResNet(bottleNeck, [3,8,36,3])


In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet.resnet50().to(device)
x = torch.randn(3,3,224,224).to(device)
output = model(x)
print(output.size())

torch.Size([3, 10])


In [30]:
summary(model, (3, 224, 224), device = device.type)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 56, 56]          --
|    └─Conv2d: 2-1                       [-1, 64, 112, 112]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 112]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 112]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 56]          --
├─Sequential: 1-2                        [-1, 256, 56, 56]         --
|    └─bottleNeck: 2-5                   [-1, 256, 56, 56]         --
|    |    └─Sequential: 3-1              [-1, 256, 56, 56]         58,112
|    |    └─Sequential: 3-2              [-1, 256, 56, 56]         16,896
|    |    └─ReLU: 3-3                    [-1, 256, 56, 56]         --
|    └─bottleNeck: 2-6                   [-1, 256, 56, 56]         --
|    |    └─Sequential: 3-4              [-1, 256, 56, 56]         70,400
|    |    └─Sequential: 3-5              [-1, 256, 56, 56]         --

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 56, 56]          --
|    └─Conv2d: 2-1                       [-1, 64, 112, 112]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 112]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 112]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 56]          --
├─Sequential: 1-2                        [-1, 256, 56, 56]         --
|    └─bottleNeck: 2-5                   [-1, 256, 56, 56]         --
|    |    └─Sequential: 3-1              [-1, 256, 56, 56]         58,112
|    |    └─Sequential: 3-2              [-1, 256, 56, 56]         16,896
|    |    └─ReLU: 3-3                    [-1, 256, 56, 56]         --
|    └─bottleNeck: 2-6                   [-1, 256, 56, 56]         --
|    |    └─Sequential: 3-4              [-1, 256, 56, 56]         70,400
|    |    └─Sequential: 3-5              [-1, 256, 56, 56]         --